# 📚 Analyse des ventes

Après un an de ventes en ligne, la société *Rester Livres* dispose de suffisamment de données pour dresser une typologie précise de ses clients et produits. Nous allons vérifier s'il existe un lien entre le sexe des clients et les catégories de produits achetés, l'âge et le montant total des achats, la fréquence d'achat ou encore la taille du panier moyen.

Hormis les librairies Pandas et Seaborn, nous utiliserons scikit-learn pour confirmer les corrélations à l'aide de tests statistiques.

## Sommaire

- 1. [Exploration](#1.-Exploration)
    - 1.1. [Clients](#1.1.-Clients)
    - 1.2. [Produits](#1.2.-Produits)
    - 1.3. [Transactions](#1.3.-Transactions)
    - 1.4. [Jointures](#1.4.-Jointures)
- 2. [Nettoyage](#2.-Nettoyage)
    - 2.1. [Lignes tests](#2.1.-Lignes-tests)
    - 2.2. [Dates manquantes](#2.2.-Dates-manquantes)
    - 2.3. [Valeurs manquantes](#2.3.-Valeurs-manquantes)
    - 2.4. [Variables supplémentaires](#2.4.-Variables-supplémentaires)
- 3. [Analyse](#3.-Analyse)
    - 3.1. [Inégalités entre clients](#3.1.-Inégalités-entre-clients)
    - 3.2. [Clients professionnels](#3.2.-Clients-professionnels)
    - 3.3. [Clients particuliers](#3.3.-Clients-particuliers)
        - 3.3.1. [Par sexe](#3.3.1.-Par-sexe)
        - 3.3.2. [Par âge](#3.3.2.-Par-âge)
    - 3.4. [Produits](#3.4.-Produits)
        - 3.4.1. [Périodicité](#3.4.1.-Périodicité)
        - 3.4.2. [Catégories et prix](#3.4.2.-Catégories-et-prix)
- 4. [Tests](#4.-Tests)
    - 4.1. [Catégorie et âge (ANOVA)](#4.1.-Catégorie-et-âge-(ANOVA))
    - 4.2. [Catégorie et sexe (Chi 2)](#4.2.-Catégorie-et-sexe-(Chi-2))
    - 4.3. [Prix et montant total des achats (Pearson)](#4.3.-Prix-et-montant-total-des-achats-(Pearson))
- 5. [Références](#5.-Références)

In [ ]:
# Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import ensemble
from datetime import datetime
import scipy.stats as stats

In [ ]:
sns.set( # Styles Seaborn
    style='whitegrid',
    context='notebook',
    palette='Paired',
    rc={'figure.figsize':(8,4)})

In [ ]:
# Affichage des nombres : séparateur des milliers, et réduction du nombre de décimales
pd.options.display.float_format = '{:,.2f}'.format

# 1. Exploration

On dispose de 3 jeux de données : **clients**, **produits** et **transactions**.

In [ ]:
url = 'https://raw.githubusercontent.com/gllmfrnr/oc/master/p4/sources/'
clients = pd.read_csv(url + 'customers.csv')
produits = pd.read_csv(url + 'products.csv')
transactions = pd.read_csv(url + 'transactions.csv')

## 1.1. Clients

Recense l'identifiant, le sexe et l'année de naissance de 8622 individus.

In [ ]:
clients

On vérifie que la table ne contient aucun doublon avant de pouvoir déterminer la clé primaire, à savoir *'client_id'* : les 8622 clients sont tous distincts.

In [ ]:
def doublons(df): # Nombre de doublons dans la dataframe
    print(len(df) - len(df.drop_duplicates()), 'doublons')

doublons(clients)

In [ ]:
def cle_primaire(df_cle): # Vérification de la clé primaire
  table_length = len(df_cle) - len(df_cle.drop_duplicates())
  if table_length == 0:
    print('Clé primaire (0 doublon)')
  else:
    print('Pas une clé primaire (', table_length, 'doublons )')
    
cle_primaire(clients['client_id'])

Les variables ne contiennent aucune valeur manquante ou aberrante.

In [ ]:
clients.info() # Valeurs manquantes et types des variables

Dans la seule variable quantitative, *'birth'* , les années de naissance s'étalent de 1929 à 2004. 

In [ ]:
clients.describe(include='all') # Indicateurs statistiques

La variable *'sex'* n'affiche elle que 2 modalités : *f* (féminin) et *m* (masculin).

In [ ]:
print('Modalités de \'sex\' :', list(clients['sex'].unique()))

## 1.2. Produits

La table **produits** détaille l'identifiant, le prix et la catégorie de 3287 livres. 

In [ ]:
produits

Là encore aucun doublon ni valeur manquante. *'Id_prod'* est la clé primaire : les produits sont uniques.

In [ ]:
doublons(produits) # Nombre de doublons dans la dataframe

In [ ]:
cle_primaire(produits['id_prod']) # Vérification de la clé primaire

In [ ]:
produits.info() # Valeurs manquantes et types des variables

La variable *'price'* compte des prix de -1 à 300. Le produit *T_0* est le seul à afficher un prix négatif de *-1*. Il faudra nettoyer cette valeur aberrante après avoir joint les 3 tables.  

In [ ]:
produits.describe(include='all') # Indicateurs statistiques

In [ ]:
produits.sort_values(by='price') # Produit affichant un prix négatif

La variable *'categ'* compte 3 modalités : 0, 1 et 2. 

In [ ]:
print('Modalités de \'categ\' :', list(produits['categ'].unique()))

## 1.3. Transactions

Le jeu des transactions détaille la date et les identifiants de la session, du client et du produit vendu.

In [ ]:
transactions

Cette fois-ci, la table compte 126 doublons : on les supprime.

In [ ]:
doublons(transactions) # Nombre de doublons
transactions.drop_duplicates(inplace=True) # Suppression des doublons
print(len(transactions), 'lignes après suppression des doublons') # Vérification du nombre de lignes

La clé primaire est composée de la date et de l'identifiant client : deux transactions peuvent survenir au même moment.

In [ ]:
cle_primaire(transactions[['client_id', 'date']]) # Vérification de la clé primaire

Nous aurons remarqué que *'client_id* et *'id_prod'* sont des clés étrangères vers **clients** et **produits**. Dans chaque clé que référence la table **transactions**, il y a moins de valeurs uniques que dans les 2 autres tables : 21 clients enregistrés n'ont pas passé commande, et 22 produits n'ont pas été vendus. Il existe toutefois un produit présent uniquement dans **transactions** : *0_2245* (103 occurences).

In [ ]:
# Comparaison d'une clé unique entre 2 dataframes
def compare_keys(variable, df1, df1_name, df2, df2_name):
    df1_keys = pd.DataFrame(df1[variable].unique()) # Projection de df1 sur la variable
    df2_keys = pd.DataFrame(df2[variable].unique()) # Projection de df2 sur la variable
       
    keys_1 = df1_keys.merge( # Clés de df2 non présentes dans df1
        df2_keys, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']  
    print('Clés de', df2_name, 'non présentes dans', df1_name + ' : ', len(keys_1))   
    
    keys_2 = df2_keys.merge(  # Clés de df1 non présentes dans df2
    df1_keys, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
    print('Clés de', df1_name, 'non présentes dans', df2_name + ' : ', len(keys_2))
    
compare_keys('client_id', clients, 'clients', transactions, 'transactions') # Comparaison de 'client_id' entre transactions et clients
print('- ' * 24)
compare_keys('id_prod', produits, 'produits', transactions, 'transactions') # Comparaison de la clé 'id_prod' entre transactions et produits

In [ ]:
# Clé de 'id_prod' uniquement présente dans transactions
transactions.drop(transactions[transactions['id_prod'].isin(produits['id_prod'])].index) 

La commande describe() nous permet une nouvelle fois de vérifier le nombre de valeurs uniques par variable. 

Il y a moins d'identifiants de session uniques que de dates : une même session peut donc contenir plusieurs dates de transactions. Chaque client peut à son tour totaliser plusieurs sessions. 

In [ ]:
transactions.describe(include='all') # Indicateurs statistiques

La table ne contient aucune valeur manquante, mais comme aperçu dans les indicateurs, la variable *'date'* affiche 74 valeurs aberrantes ayant pour préfixe *test_*. Ces 74 individus concernent le produit au prix négatif (*T_0*), ainsi que des identifiants de session et de clients uniques là aussi. 

In [ ]:
transactions_test = transactions.sort_values(by='date', ascending=False).head(75).reset_index() # Les 75 premiers individus triés par date
transactions_test

In [ ]:
transactions_test = transactions_test.loc[:73] # Les 74 lignes de test

print( # Affichage des clés uniques des lignes de test 
    len(transactions_test), 'lignes tests :',
    '\n- \'id_prod\' unique :', transactions_test['id_prod'].unique(),
    '\n- \'session_id\' unique :', transactions_test['session_id'].unique(),
    '\n- \'client_id\' uniques :', transactions_test['client_id'].unique())

La variable *'date'* est dans le format *object* à cause de cette valeur, alors qu'elle devrait être en *datetime*.

In [ ]:
transactions.info() # Valeurs manquantes et types des variables

## 1.4. Jointures

On effectue la jointure de **transactions** successivement sur **clients** et **produits**, en ne conservant que les clés de **transactions** (les clients inactifs et les produits invendus sont écartés).

La clé primaire reste celle de **transactions** : *'date'* + *'client_id'*.

In [ ]:
df = transactions.merge(clients, how='left', on='client_id').merge(produits, how='left', on='id_prod') # Jointures
df

In [ ]:
cle_primaire(df[['client_id', 'date']]) # Vérification de la clé primaire

Les variables *'price'* et *'categ'* affichent un même nombre de valeurs manquantes, qui concernent encore une fois le produit *0_2245* (uniquement présent dans **transactions** à l'origine). On cherchera plus loin à remplacer ces valeurs avec la méthode la mieux adaptée.

In [ ]:
df.info() # Valeurs manquantes

In [ ]:
df[(df['price'].isnull()) & (df['categ'].isnull())] # Individus sans prix ni catégorie

# 2. Nettoyage

## 2.1. Lignes tests

Avant de pouvoir convertir la variable *'date'* en datetime, il faut supprimer les lignes de test, vu qu'elles partagent notamment le préfixe *test_* en date.

In [ ]:
lignes_test = df[ # Échantillon des lignes de test
    (df['date'].str.contains('test_')) &
    (df['price']<=0) &
    (df['id_prod']=='T_0') &  
    (df['session_id']=='s_0')]
lignes_test

En supprimant ces 74 lignes de tests, on débarasse la table de ses valeurs aberrantes (*-1*), du préfixe *test_* dans les dates, ainsi que des faux clients et sessions.

In [ ]:
len_df_before = len(df) # Nombre de lignes dans df avant suppression des lignes tests
df = df.drop(lignes_test.index).reset_index() # Suppression des lignes tests
print(len_df_before - len(df), 'lignes tests supprimées') # Nombre de lignes dans data après nettoyage

La clé primaire devient *'date'* : il y a maintenant autant de transactions que de dates.

In [ ]:
cle_primaire(df['date']) # Vérification de la clé primaire

## 2.2. Dates manquantes

On peut maintenant convertir la variable *'date'* en datetime. 

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d') # Conversion de 'date' en datetime
df.info()

Les transactions s'étalent sur un an, du 1er mars 2021 au 28 février 2022.

In [ ]:
print('Transactions du', str(df['date'].min())[:10], 'au', str(df['date'].max())[:10]) # Dates de transaction la plus ancienne et la plus récente

L'histogramme des dates par catégorie de produits montre qu'il manque des données au mois d'octobre dans la catégorie 1.

In [ ]:
# Volume des ventes par date et catégorie
plt.figure(figsize=(12,8))
sns.histplot(data=df, x='date', hue='categ', palette=["#ff9e80", "#ff6e40", "#ff3d00"])
plt.xticks(rotation=45)
plt.show()

Pour équilibrer l'analyse on supprime toutes les données du mois d'octobre, soit 6% du dataset.

In [ ]:
len_df_before = len(df) # Nombre de lignes avant la suppression du mois d'octobre

# Transactions du mois octobre, toutes catégories
octobre = df[(df['date']>='2021-10-01') & (df['date']<='2021-10-31')]

df.drop(octobre.index, inplace=True) # Suppression des données du mois d'octobre

# Pourcentage du dataset écarté
print('Mois d\'octobre = ', round(((len_df_before - len(df)) / len_df_before * 100), 1), '% du dataset écarté')

In [ ]:
# Volume des ventes par date et catégorie (sans le mois d\'octobre
plt.figure(figsize=(12,8))
sns.histplot(data=df, x='date', hue='categ', palette=["#ff9e80", "#ff6e40", "#ff3d00"])
plt.xticks(rotation=45)
plt.show()

## 2.3. Valeurs manquantes

Reste à choisir une méthode pour traiter les valeurs manquantes du produit *0_2245*, à savoir son prix et sa catégorie.

Pour gérer les valeurs manquantes, 3 méthodes sont envisageables :
- supprimer les lignes concernées
- déterminer une valeur fixe
- modéliser cette valeur

In [ ]:
produit_2245 = df[df['id_prod']=='0_2245'] # Échantillon du produit 0_2245
produit_2245

Ici l'échantillon ne représente que 0.03% du dataset : on pourrait le supprimer sans fausser l'analyse.

In [ ]:
# Pourcentage des ventes du produit 0_2245
print('Produit 0_2245 :', round((len(produit_2245) / len(df) * 100), 2), '% du dataset')

On peut également déterminer une valeur fixe pour les 2 variables. Pour la catégorie, on peut se fier aux préfixes des identifiants de produits : *0_*, *1_* et *2_*. Ces préfixes correspondent invariablement à la catégorie du produit concerné. On choisit donc *0* comme catégorie du produit *0_2245*.

In [ ]:
df_test = df.dropna() # Échantillon de l'ensemble des commandes, sans le produit 0_2245

# Conversion en string des 2 premiers caractères de la valeur de 'id_prod'
df_test['id_prod'] = df_test['id_prod'].str[:2]

print('Préfixes de \'id_prod\' :') # Pour chaque catégorie, le préfixe unique de 'id_prod'
for i in df_test['categ'].unique():
    print('- catégorie', i, ':',
        df_test[df.dropna()['categ']==i]['id_prod'].unique())

In [ ]:
# Remplacement des valeurs manquantes de la variable 'categ' par 0
df['categ'].replace(np.nan, 0, inplace=True)
produit_2245 = df[df['id_prod']=='0_2245']
produit_2245.sample(3)

Comme le démontrera l'analyse, les 3 catégories correspondent à des tranches de prix ordonnées. Les indices de corrélation confirment que cette variable des catégories est la plus à même d'expliquer le prix. Dans une moindre mesure, l'âge semble également avoir un impact sur le prix.

In [ ]:
# Triangle de corrélations
plt.figure(figsize=(8, 5))
heatmap = sns.heatmap(df.corr(), mask=np.triu(np.ones_like(df.corr(), dtype=np.bool)), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle de corrélations', fontdict={'fontsize':16}, pad=24)
plt.xticks(rotation=45)
plt.show()


On pourrait donc déterminer le prix en se basant sur les indicateurs de prix dans la catégorie 0. La moitié des produits y ont un prix supérieur à 9.99 : on peut choisir ce prix médian comme valeur de remplacement. On aurait également pu retenir le prix le plus fréquent (le mode) de la catégorie 0 : *4.99*.

In [ ]:
# Distribution et moyenne des prix par catégorie
plt.figure(figsize=(8,3))
sns.boxplot(data=df, y='categ', x='price', orient='h', showfliers=False, showmeans=True, palette=["#64ffda", "#536dfe", "#ff6e40"], meanprops={"marker":"s","markerfacecolor":"white"})
plt.show()

In [ ]:
print('Catégorie 0 :',
    '\n- Prix moyen :', round(df[df['categ']==0]['price'].mean(), 2), # Prix moyen
    '\n- Prix médian :', df[df['categ']==0]['price'].median(), # Prix médian
    '\n- Mode :', df[df['categ']==0]['price'].mode().values[0]) # Prix le plus fréquent

Pour trancher entre ces 2 prix (la médiane et le mode), on peut alors s'appuyer sur une régression linéaire pour modéliser le prix probable du produit *0_2245*. On retient comme variables explicatives les colinéarités relevées plus tôt.

Les prédictions indiquent des prix moyen et médian proches de 9.99. Le mode s'en approche également, on retient donc 9.99 comme valeur de remplacement.

In [ ]:
train = df[df['id_prod']!='0_2245'] # Échantillon d'entrainement 
y = train['price'].to_numpy() # Variable cible
features = ['birth', 'categ'] # Variables explicatives
X = pd.get_dummies(train[features]).to_numpy() # Échantillon d'entrainement (variables encodées)
X_test = pd.get_dummies(produit_2245[features]).to_numpy() # Échantillon test
model = sk.linear_model.LinearRegression() # Modèle
model.fit(X, y) # Fit les variables explicatives et la target sur le modèle
predictions = pd.Series(model.predict(X_test)) # Prédiction sur l'échantillon test

plt.figure(figsize=(8, 2))
sns.violinplot(data=predictions, orient='h', showfliers=False, showmeans=True)
sns.boxplot(data=predictions, orient='h', showfliers=False, showmeans=True)
plt.title('Distribution et moyenne des prédictions de prix du produit 0_2245')
plt.show()

In [ ]:
print('Prédictions sur la catégorie:',
    '\n- Prix moyen :', round(predictions.mean(), 2),
    '\n- Prix médian :', round(predictions.median(), 2),
    '\n- Mode :', round(predictions.mode(), 2)[0])

In [ ]:
df['price'].replace(np.nan, 9.99, inplace=True)

Une forêt d'arbres décisionnels aurait aussi pu nous confirmer la catégorie. On prend comme explicatives les 2 seules variables à disposition dans l'échantillon du produit (le sexe et l'âge). La majorité des prédictions classent le produit dans la catégorie 0.

In [ ]:
train = df[df['id_prod']!='0_2245']
features = ['sex', 'birth']
y = train['categ']
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(produit_2245[features])
model = sk.ensemble.RandomForestClassifier()
model.fit(X, y)
predictions = model.predict(X_test)

sns.countplot(predictions)
plt.title('Prédictions de catégorie des individus du produit 0_2245')
plt.show()

## 2.4. Variables supplémentaires

Pour enrichir l'analyse, on crée quelques variables additionnelles, comme : 

- le mois de la transaction
- le nombre de produits achetés par client chaque mois (la fréquence d'achats)
- le nombre de ventes total par client sur l'année entière
- le panier moyen, 
- la taille du panier moyen pour chaque client
- le chiffre d'affaires total par client sur l'année

In [ ]:
# Mois de la transaction
df['mois'] = pd.DatetimeIndex(df['date']).month # Mois, de 1 à 12
df.sample()

df['date_fixe'] = df['date'].dt.date # Variable temporaire de date fixe (jour)

df = df.merge(
    df.groupby('client_id').count()['date'].reset_index().rename(columns={'date': 'total_ventes'}),
    how='left', on='client_id')
df.sample(3)

df['ventes_mensuelles'] = round(df['total_ventes'] / 11)
df.sample()

df = df.merge(
    df.pivot_table(
        index=['client_id', 'date_fixe'], 
        values='price', 
        aggfunc='count').reset_index().pivot_table(
        index='client_id').reset_index().rename(
        columns={'price': 'taille_panier_moyen'}), 
    on='client_id', how='left')

df = df.merge(
    df.pivot_table(
        index=['client_id', 'date_fixe'], 
        values='price').reset_index().pivot_table(
        index='client_id').reset_index().rename(
        columns={'price': 'panier_moyen'}), 
    on='client_id', how='left').drop('date_fixe', axis=1)


df = df.merge(
    df.pivot_table(
    index='client_id', values='price', 
    aggfunc='sum').reset_index().rename(
    columns={'price': 'total_achats'}),
    on='client_id', how='left')
df.sample(3)

On peut également diviser la population en tranches d'âges, par tranches de 10 ans.

In [ ]:
year = datetime.now().year # Année courante
df['age'] = year - df['birth'] # Âge du client
df.sample()

# Tranches d'âge
df['classe_age'] = '18-30'
df['classe_age'].loc[df[df['age']>=30].index] = '30-40'
df['classe_age'].loc[df[df['age']>=40].index] = '40-50'
df['classe_age'].loc[df[df['age']>=50].index] = '50-60'
df['classe_age'].loc[df[df['age']>=60].index] = '60-70'
df['classe_age'].loc[df[df['age']>=70].index] = '70-80'
df['classe_age'].loc[df[df['age']>=80].index] = '80 et +'
df.sample()

# 3. Analyse

## 3.1. Inégalités entre clients

En triant les clients par chiffre d'affaires annuel, 4 clients se détachent. Leur nombre d'achats est anormalement plus élevé que celui des autres clients : on a sûrement affaire à des clients professionels.

In [ ]:
ca_annuel = df.pivot_table(
    index='client_id', values=[
        'total_achats','ventes_mensuelles','taille_panier_moyen','total_ventes','panier_moyen']
    ).sort_values(by='total_achats', ascending=False).reset_index()

ca_annuel.head(10) # Les 10 clients aux plus gros chiffres d'affaires annuels

Le poids de ces 4 individus risquant d'impacter l'analyse sur des variables comme le sexe ou l'âge, on distinguera les clients professionnels des particuliers.  

In [ ]:
# Création d'une variable 'Type' de client (2 modalités : particulier ou professionnel)
df['client_type'] = 'B2C'
df['client_type'].loc[df[df['client_id'].isin(ca_annuel.head(4)['client_id'])].index] = 'B2B'

# Création de 2 datafra
b2b = df[df['client_type']=='B2B']
b2c = df[df['client_type']=='B2C']

df.sample(3)

Le B2B représente 7.5% du chiffre d'affaires, et 6.9% des transactions.

In [ ]:
print('Clients professionnels :',
    round(b2b['price'].sum() / df['price'].sum() * 100, 2), '% du chiffre d\'affaires annuel')

In [ ]:
# Proportion des transactions entre professionnels et particuliers
plt.figure(figsize=(5,5))
df['client_type'].value_counts(normalize=True).plot(
    kind='pie',
    legend=True,
    autopct='%1.1f%%')
plt.show()

L'indice de Gini mesure ici l'inégalité des chiffres d'affaires de tous les clients : 

- un coefficient de 0 indiquerait une égalité parfaite
- si l'indice était de 1, un seul client détiendrait la totalité du chiffre d'affaires

La courbe de Lorenz montre encore une fois le chiffre d'affaires que les 4 clients (représentés par des croix) représentent à eux-seuls. 

In [ ]:
# Courbe de Lorenz
def lorenz(variable):
    X = variable.values
    X = np.sort(X)
    
    # Indice de Gini
    def gini(array):
        array
        sorted_array = array.copy()
        sorted_array.sort()
        n = array.size
        coef_ = 2. / n
        const_ = (n + 1.) / n
        weighted_sum = sum([(i+1)*yi for i, yi in enumerate(sorted_array)])
        return coef_*weighted_sum/(sorted_array.sum()) - const_
    print('Incide de Gini :', gini(X))
    
    # Courbe de Lorenz
    X_lorenz = X.cumsum() / X.sum()
    X_lorenz = np.insert(X_lorenz, 0, 0)
    # X_lorenz[0], X_lorenz[-1]
    y = np.arange(X_lorenz.size)/(X_lorenz.size-1)
    lorenz = pd.DataFrame()
    lorenz['X'] = pd.Series(X_lorenz)
    lorenz['Y'] = pd.Series(y)
    sns.scatterplot(data=lorenz, x='Y', y='X', marker='x')

    # Diagonale
    a = np.arange(0,1,.01)
    x = a
    y = a
    
    # Graphique
    sns.lineplot(x,y)
    plt.xlim([0,1])
    plt.ylim([0,1])
    
# Inégalité des chiffres d'affaires annuels de tous les clients
lorenz(ca_annuel['total_achats'])

L'indice de Gini est naturellement plus faible si on exclut les 4 professionnels.

In [ ]:
# Inégalité des chiffres d'affaires annuels des clients particuliers
lorenz(b2c.pivot_table(index='client_id')['total_achats'])

## 3.2. Clients professionnels

Les variables *'sex'* et *'age'* n'ont aucun intérêt ici. Nous remarquons que le client *c_4958* a un panier moyen supérieur aux 3 autres professionnels, alors que sa fréquence d'achat est la plus faible.

In [ ]:
b2b.pivot_table(index='client_id').reset_index() # Profil des 4 clients B2B

Le volume des transactions par catégorie montre des habitudes différentes chez chacun de ces clients. 

Le client *c_1609* ne consomme pas du tout la catégorie 2, tandis que le client *c_4858* est le seul à consommer principalement cette catégorie. Vu que ce client détient de loin le plus gros panier moyen, la catégorie pourrait influer sur le prix du livre. 

In [ ]:
sns.countplot(data=b2b, x='client_id', hue='categ')
plt.title('Volume des achats des clients professionnels, par catégorie')
plt.show()

La distribution des prix d'achats confirme cette intuition : le client *c_4958* achète des livres bien plus chers les 3 autres clients.

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=b2b, y='price', x='client_id', showfliers=False)
plt.title('Répartition des prix d\'achat par client B2B et catégorie')
plt.show()

La périodicité des achats présente des points communs en fonction des catégories là encore :

- la catégorie 0 est surtout achetée en septembre
- la catégorie 1 en décembre
- la catégorie 2 au mois de février

In [ ]:
for i in b2b['client_id'].unique():
    print('Client', i)
    plt.figure(figsize=(8,5))
    sns.histplot(data=b2b[b2b['client_id']==i], x='date', hue='categ', binwidth=7)
    plt.show()

## 3.3. Clients particuliers

### 3.3.1. Par sexe

La pyramide des âges ne montre aucune différence entre les hommes et les femmes.

In [ ]:
fig, axes = plt.subplots(1, 2)
fig.suptitle('Pyramide des âges par sexe')

sns.histplot(ax=axes[0], y=b2c[b2c['sex']=='m']['age'], bins=12)
axes[0].invert_xaxis()
axes[0].set_title('Hommes')

sns.histplot(ax=axes[1], y=b2c[b2c['sex']=='f']['age'], bins=12)
axes[1].set_yticklabels([])
axes[1].set_ylabel('')
axes[1].set_title('Femmes')

plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

La périodicité des ventes est la même pour les 2 sexes.

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=b2c, x='date', hue='sex', binwidth=7)
plt.title('Volume des ventes par sexe et date d\'achat')
plt.show()

Les effectifs de ventes pour chaque sexe sont équilibrés. Les catégories y apparaissent dans les mêmes proportions.

In [ ]:
sns.displot(data=b2c, x='sex', hue='categ')
plt.title('Effectifs des ventes B2C par sexe et catégorie')
plt.show()

La distribution des prix est là aussi la même : toutes les corrélations ont été étudiées, le sexe ne nous renseigne pas sur les habitudes des clients.

In [ ]:
plt.figure(figsize=(8,3))
sns.violinplot(data=b2c, x='price', y='sex', showfliers=False, showmeans=True)
plt.xlim([0,35])
plt.show()

Une matrice des corrélations aurait pu nous montrer cette absence de relation entre le sexe et toutes les autres variables. On doit d'abord encoder la variable *'sex'*. Les taux de corrélation approchent tous de 0, indiquant une absence de corrélation.

In [ ]:
b2c = pd.concat([b2c,
    pd.get_dummies(b2c['sex'], drop_first=True).rename(columns={'m': 'sex_code'})],
    axis=1)
b2c.sample(3)

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(
    b2c.corr(), 
    mask=np.triu(np.ones_like(b2c.corr(), dtype=np.bool)), 
    vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle correlation heatmap', fontdict={'fontsize':16}, pad=16)
plt.show()

### 3.3.2. Par âge

Les 30-50 ans représentent plus de la moitié des ventes.

In [ ]:
sns.boxplot(data=b2c, x='age', showfliers=False)
sns.violinplot(data=b2c, x='age', showfliers=False)
plt.title('Distribution des âges')
plt.show()

Les 3 catégories sont consommées par toutes les classes d'âge. Mais les acheteurs de la catégorie 0 sont principalement ces 30-50 ans, tandis que la catégorie 2 est consommée quasi exclusivement par les moins de 30 ans.

In [ ]:
plt.figure(figsize=(8,5))
sns.displot(data=b2c, x='age', hue='categ', kind='kde', fill=True)
plt.show()

Comme c'était le cas avec un des clients professionnels, les moins de 30 ans affichent des prix d'achat bien plus hauts que les autres clients.

Les prix sont aussi légèrement inférieurs chez les 30-50 ans (population consommant la catégorie 0). 

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(data=df.sort_values(by='classe_age'), y='classe_age', x='price', showfliers=False)
plt.show()

Le chiffre d'affaires annuel des clients particuliers est logiquement corrélé à la classe d'âge, car comme vu plus tôt :

- la catégorie 0 est de loin la catégorie la plus consommée (par les 30-50 ans)
- la catégorie 2 est presque exclusivement achetée par les moins de 30 ans, en plus de concentrer les livres les plus chers

Les 30-50 ans concentrent ainsi les plus gros chiffres d'affaires, suivis par les moins de 30 ans.

In [ ]:
sns.boxplot(data=b2c.sort_values(by='classe_age'), 
            y='classe_age', x='total_achats', 
            showfliers=False, showmeans=True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.scatterplot(data=b2c.sample(150), x='age', y='total_achats', hue='classe_age', s=100)
plt.title('Distribution des prix par tranche d\'âge')
plt.show()

La fréquence d'achats (le nombre de livres par mois) classe d'autant mieux les 3 groupes d'individus définis plus tôt :

- les moins de 30 ans achètent le même nombre de livrees (jamais plus de 3 livres par mois)
- les 30-50 ans se démarquent : un quart de ces clients achète plus de 8 livres par mois.
- les plus de 50 ans ont des habitudes plus variées, mais achètent rarement plus de 5 livres

In [ ]:
plt.figure(figsize=(16,4))
sns.boxplot(
    data=b2c.pivot_table(index=['client_id']),
    y='ventes_mensuelles', x='age', showfliers=False)
plt.xticks(rotation=90)
plt.title('Distribution des fréquences d\'achat, par âge')
plt.show()

La taille du panier moyen suit la même logique : 

- les moins de 30 ans comptent en moyenne moins de 2 livres par panier
- les 30-50 ans achètent 2 à 3 livres par commande
- les habitudes sont aléatoires chez les plus de 50 ans

In [ ]:
plt.figure(figsize=(16,4))
sns.boxplot(
    data=b2c.pivot_table(index=['client_id']),
    y='taille_panier_moyen', x='age', showfliers=False, showmeans=True)
plt.xticks(rotation=90)
plt.title('Distribution de la taille du panier moyen, par âge')
plt.show()

[](http://)

## 3.4. Produits

### 3.4.1. Périodicité

Comme chez les clients professionnels, la périodicité des ventes semble plutôt corrélée à la catégorie des livres :
    
- la catégorie 0 à la rentrée scolaire de septembre
- la catégorie 1 pendant les fêtes de fin d'année
- la catégorie 2 pendant l'été et au mois de février (début du second semestre universitaire)

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(data=b2c, x='date', hue='categ', multiple='stack', binwidth=7)
plt.xticks(rotation=45)
plt.show()

In [ ]:
for i in sorted(df['categ'].unique()):
    print('Catégorie', i)
    sns.histplot(data=df[df['categ']==i], x='date')
    plt.show()

### 3.4.2. Catégories et prix

La catégorie 0 représente 60% des ventes, la catégorie 2 seulement 5%. Malgré tout, les prix des catégories 2 et 1 entraînent une inégalité des chiffres d'affaires annuels par produit. 

In [ ]:
df['categ'].value_counts(normalize=True).plot(
    kind='pie',
    legend=True,
    autopct='%1.1f%%'
)
plt.title('Volume des ventes par catégorie')
plt.show()

Les livres aux plus hauts chiffre d'affaires appartiennent tous aux catégories 1 et 2.

In [ ]:
# Les 20 livres aux plus gros chiffres d'affaires annuels
df.pivot_table(index=['id_prod', 'categ'], aggfunc={'price': np.sum}).sort_values(by='price', ascending=False).reset_index().head(10)

L'indice de Gini est plus élevé encore que celui des chiffres d'affaires des clients.

In [ ]:
# Inégalités des chiffres d'affaires annuels par produit
lorenz(df.pivot_table(index='id_prod', aggfunc='sum')['price'])

Les proportions du chiffre d'affaires sont beaucoup plus équilibrées : la catégorie 0 ne représente finalement qu'un tiers du chiffre d'affaires.

In [ ]:
data=df.pivot_table(index='categ', aggfunc={'price': np.sum}).plot(
    kind='pie', y='price',
    legend=True,
    autopct='%1.1f%%'
)
plt.title('Chiffre d\'affaires par catégorie')
plt.show()

Les indicateurs prix sont distinctement ordonnés entre catégories.  

In [ ]:
plt.figure(figsize=(8,3))
sns.boxplot(
    data=df, y='categ', x='price', 
    showmeans=True, showfliers=False, orient='h')
plt.title('Distribution des prix par catégorie')
plt.show()

In [ ]:
# Indicateurs de position des prix dans chaque catégorie
for i in sorted(df['categ'].unique()):
    print('Catégorie', i)
    print(pd.Series(df[df['categ']==i]['price'].describe()), '\n')

# 4. Tests

## 4.1. Catégorie et âge (ANOVA)

Entre une catégorielle et une quantitative, L'analyse de la variance.

Avant de faire une ANOVA, on affiche la moyenne des âge dans chaque catégorie. La catégorie 2 se détache, mais les catégories 0 et 1 ont des moyennes assez proches : le test va permettre de vérifier si leur distance est significative.

In [ ]:
sns.boxplot(data=df, x='age', y='categ', orient='h', showfliers=False, showmeans=True)
plt.show()

In [ ]:
# Moyenne des prix par catégorie
df.groupby('categ').mean()['age']

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

# Test d'ANOVA
sample_df = df.sample(5000)
model = smf.ols('age ~ categ', data=sample_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
p = anova_table['PR(>F)'][0]
print('ANOVA\np-value :', p, '\nstat :', anova_table['F'][0])
if p > 0.05:
    print('H0: the means of the samples are equal.')
else:
    print('H1: one or more of the means of the samples are unequal.'
         '\n\nConditions :'
         '\n1. Normalité des résidus\n2. Homoscédasticité')

Mais la distribution des résidus n'est pas normale.

In [ ]:
from statsmodels.graphics.gofplots import qqplot

# Test de Shapiro sur les résidus
print('Normalité des résidus (Shapiro)')
print('\nstats :', stats.shapiro(model.resid)[0],
    '\np-value :', stats.shapiro(model.resid)[1])
if p>.05:
    print('H0 acceptée : distribution normale')
else:
    print('H0 rejetée : distribution probablement pas normale')
qqplot(model.resid, line='s')
plt.show()

Après un boxcox les résidus sont considérés comme normaux : la première condition de l'ANOVA est remplie.

In [ ]:
from scipy.special import boxcox1p

# Test de Shapiro sur les résidus après boxcox
residus = boxcox1p(model.resid, 1)
p = stats.shapiro(residus)[1]
print('Normalité des résidus après boxcox :'
    '\nstats :', stats.shapiro(residus)[0],
    '\np-value :', p)
if p>.05:
    print('H0 acceptée : distribution normale')
else:
    print('H0 rejetée : distribution probablement pas normale')
qqplot(residus, line='s')
plt.show()

Reste à tester l'homoscédasticité des résidus, pour s'assurer que les variances sont égales. Le test de Levene n'est pas concluant. Dans ce cas où la condition d'homoscédasticité n'est pas remplie, on peut utiliser Welch ANOVA : ce test valide la corrélation entre prix et catégorie.

In [ ]:
# Test de Levene sur les 3 catégories
samples = 4000
a = df[df['categ']==0]['age'].sample(samples).values
b = df[df['categ']==1]['age'].sample(samples).values
c = df[df['categ']==2]['age'].sample(samples).values
stat, p = stats.levene(a, b, c)
print('Condition 2 : homoscédasticité (Levene)',
    '\nstats :', stat,
    '\np-value :', p)
if p > 0.05:
    print('H0: les variances sont égales')
else:
    print('H1: les variances ne sont pas égales (essayer Welch ANOVA)')
print('\nConditions :'
      '\n- The samples from the populations under consideration are independent',
      '\n- The populations under consideration are approximately normally distributed')   

In [ ]:
# Test de Welch entre catégorielle et quantitative
stat, p = stats.ttest_ind(sample_df['age'], sample_df['categ'])
print('Test de Welch (si absence d\'homoscédasticité)\n')
print('p-value :', p, '\nstat :', stat)
if p > 0.05:
    print('H0: les moyennes des échantillons sont égales')
else:
    print('H1: une ou plus des moyennes des échantillons sont inégales')

## 4.2. Catégorie et sexe (Chi 2)

In [ ]:
sns.displot(data=b2c, x='sex', hue='categ')
plt.title('Effectifs des ventes B2C par sexe et catégorie')
plt.show()

In [ ]:
pip install researchpy

In [ ]:
# Table de contingence
crosstab = pd.crosstab(b2c['categ'], b2c['sex'], margins=False)
print('Table de contingence :')
print(crosstab)

import researchpy as rp

# Table de contingence normalisée
table, results = rp.crosstab(b2c['categ'], b2c['sex'], prop='col', test='chi-square')
print('\n' + '-' * 55, '\n\nTable de contingence normalisée :')
print(table)

In [ ]:
sns.heatmap(crosstab)
plt.show()

In [ ]:
# Test de Chi 2
sample_df = b2c.sample(1500)
table = pd.crosstab(sample_df['categ'], sample_df['sex'], margins=False)
stat, p, dof, expected = stats.chi2_contingency(table)
print('Test de Chi 2 (2 catégorielles)')
print('Stat = %.3f\np-value = %.35f' % (stat, p))
if p > .05:
    print('H0: the two samples are independent')
else:
    print('H1: there is a dependency between the samples.')
print('\nConditions :\n'
      '- Observations used in the calculation of the contingency table are independent.',
      '\n- 25 or more examples in each cell of the contingency table.')

## 4.3. Prix et montant total des achats (Pearson)

In [ ]:
sns.scatterplot(data=b2c, x='price', y='total_achats', hue='categ')
plt.show()

In [ ]:
df.sample()

In [ ]:
# Test de Pearson
def test_pearson(samples, data, variable1, variable2):
    df = data.sample(samples)
    stat, p = stats.pearsonr(df[variable1], df[variable2])
    print('Test de Pearson (2 quantitatives)')
    print('Stat = %.3f\np-value = %.35f' % (stat, p))
    if p > .05:
        print('H0: Dépendances entre les échantillons')
    else:
        print('H1: Pas de dépendance entre les échantillons')
    print('\nConditions :\n'
          '- Observations in each sample are independent and identically distributed (iid)',
          '\n- Observations in each sample are normally distributed',
          '\n- Observations in each sample have the same variance')
    sns.regplot(data=df, x=variable1, y=variable2)
    plt.show()
    
test_pearson(1000, b2c, 'price', 'total_achats')

# 5. Références

Statistiques :
- [StatQuest - Statistics Fundamentals](https://www.youtube.com/playlist?list=PLblh5JKOoLUK0FLuzwntyYI10UQFUhsY9)

Modélisation : 
- [Getting Started With Titanic](https://www.kaggle.com/alexisbcook/getting-started-with-titanic) (Kaggle)
- [Stackoverflow](https://stackoverflow.com/questions/41925157/logisticregression-unknown-label-type-continuous-using-sklearn-in-python)

Courbe de Lorenz :
- [Plot Lorenz Curve in Python](https://zhiyzuo.github.io/Plot-Lorenz/)
- [Clearly Explained: Gini coefficient and Lorenz curve](https://towardsdatascience.com/clearly-explained-gini-coefficient-and-lorenz-curve-fe6f5dcdc07)  

Tests :
- [17 Statistical Hypothesis Tests in Python (Cheat Sheet)](https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/)
- [Independent and identically distributed random variables](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)
- [Q-Q Plots Explained](https://medium.com/preai/q-q-plots-explained-75f5bc6d68be)


ANOVA :
- [ANOVA explained without math: Why analyze variances to compare means?](https://medium.com/@peterflom/anova-why-analyze-variances-to-compare-means-e3d4bbd3c05)
- [ANOVA using Python](https://reneshbedre.github.io/blog/anova.html)
- [F-tests and ANOVAs — Examples with the Iris dataset](https://medium.com/@rrfd/f-tests-and-anovas-examples-with-the-iris-dataset-fe7caa3e21d0)
- [Welch’s t-test](https://pythonfordatascienceorg.wordpress.com/welch-t-test-python-pandas/)